In [ ]:
# Python extension for interfacing with SQL and better table formatting

#!pip install ipython-sql
#!pip install pandas

In [4]:
# Necessary in the Jupyter to load the SQL extension and connect to the database file, currently using SQLite.
# Also formatting the SQL query outputs into a better format

%load_ext sql
%sql sqlite:///chinook.db
%config SqlMagic.autopandas=True

In [7]:
%%sql --quiet
SELECT tracks.Name AS `Song`, albums.Title AS `Album`, genres.Name 
AS `Genre`, SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS Revenue
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN genres ON tracks.GenreId = genres.GenreId
JOIN invoice_items ON tracks.TrackId = invoice_items.TrackId
GROUP BY tracks.TrackId
ORDER BY revenue DESC
LIMIT 5;

 * sqlite:///chinook.db
Done.


,Song,Album,Genre,Revenue
0,The Woman King,"Battlestar Galactica, Season 3",Science Fiction,3.98
1,The Fix,"Heroes, Season 1",Drama,3.98
2,Walkabout,"Lost, Season 1",TV Shows,3.98
3,Hot Girl,"The Office, Season 1",TV Shows,3.98
4,Gay Witch Hunt,"The Office, Season 3",TV Shows,3.98
